Mounting the drive to access the data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn 
import plotly.express
import xgboost as xgb
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

Loading the dataset

In [3]:
ratings_path = '/content/drive/MyDrive/Big_Data/ratings.csv'
movies_path = '/content/drive/MyDrive/Big_Data/movies.csv'
ratings = pd.read_table(ratings_path ,delimiter=',',dtype = {0:int, 1:int, 2:float, 3: int})
movies = pd.read_csv(movies_path)

In [4]:
x= movies['genres'].unique()
y = "|".join(x).split('|')
z = [*set(y)]
z

['Action',
 'Adventure',
 'Documentary',
 'Comedy',
 'Sci-Fi',
 'Romance',
 'Fantasy',
 'Thriller',
 'Animation',
 'Musical',
 'War',
 '(no genres listed)',
 'IMAX',
 'Western',
 'Crime',
 'Mystery',
 'Drama',
 'Horror',
 'Children',
 'Film-Noir']

In [5]:
len(ratings)

25000095

In [6]:
df = ratings.groupby('movieId').filter(lambda x : len(x)>1000)

In [7]:
df['movieId'].value_counts()

356     81491
318     81482
296     79672
593     74127
2571    72674
        ...  
3955     1003
5385     1002
6902     1002
2514     1001
1993     1001
Name: movieId, Length: 3790, dtype: int64

In [8]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [9]:
movies = pd.read_csv(movies_path)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
movies[movies.movieId==2268]
ratings[ratings.userId == 87701]

,userId,movieId,rating,timestamp
13551530,87701,1,4.5,1569293345
13551531,87701,150,4.0,1568938738
13551532,87701,153,1.0,1568938527
13551533,87701,161,4.5,1568938876
13551534,87701,260,4.0,1568941079
...,...,...,...,...
13551594,87701,195159,4.0,1569293539
13551595,87701,195305,4.0,1568857216
13551596,87701,197207,3.5,1568938687
13551597,87701,200598,4.0,1568857271


In [11]:
ratings_df = ratings.sample(n=10000)
ratings_df 

,userId,movieId,rating,timestamp
18310906,118616,377,2.0,966379335
12321103,79790,1288,5.0,973823863
16477621,106888,51255,5.0,1264913108
13551556,87701,2268,4.0,1568939543
20598076,133982,628,3.0,1107863981
...,...,...,...,...
719916,4896,5872,3.0,1370142062
7329658,47508,552,3.0,840017672
7546177,48993,1293,4.0,862913360
15512940,100480,1711,4.0,994801945


In [12]:
# Compute the movie similarity matrix using cosine similarity
movie_ratings_df = ratings_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
movie_similarity_matrix = cosine_similarity(movie_ratings_df.T)
movie_similarity_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [13]:
movie_ratings_df

movieId,1,2,3,4,5,6,7,8,9,10,...,201811,203244,203926,204188,204528,204630,204698,205383,205403,205501
userId,,,,,,,,,,,,,,,,,,,,,
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Random Walk Algorithm

In [15]:
# Define the random walk function
def random_walk(user, movie_similarity_matrix, alpha=0.85, max_iterations=100):
    # Initialize the scores for all movies
    scores = np.zeros(movie_similarity_matrix.shape[0])
    # Set the starting movie to be the one with the highest rating by the user
    user_ratings = movie_ratings_df.loc[user]
    starting_movie = user_ratings.idxmax()
    starting_movie_index = movie_ratings_df.columns.get_loc(starting_movie)
    scores[starting_movie_index] = 1.0
    # Perform the random walk
    for i in range(max_iterations):
        new_scores = alpha * np.dot(movie_similarity_matrix, scores) + (1 - alpha) * scores
        new_scores[starting_movie_index] = 0
        scores = new_scores
    # Return the top recommended movies
    recommended_movies = movie_ratings_df.columns[np.argsort(-scores)]
    return recommended_movies

# Generate movie recommendations for a given user
user = 87701
recommended_movies = random_walk(user, movie_similarity_matrix)
print(f"Recommended movies for {user}: {list(recommended_movies)[:10]}")

Recommended movies for 87701: [6981, 7924, 30810, 7371, 4776, 78174, 4019, 170, 58418, 65230]


Name of movies recommended


In [20]:
print("Recommended Movies are : ")
for i in range(10):
  print(movies[movies.movieId == list(recommended_movies)[i]]['title'])

Recommended Movies are : 
6856    Ordet (Word, The) (1955)
Name: title, dtype: object
7536    Stray Dog (Nora inu) (1949)
Name: title, dtype: object
9441    Life Aquatic with Steve Zissou, The (2004)
Name: title, dtype: object
7246    Dogville (2003)
Name: title, dtype: object
4670    Training Day (2001)
Name: title, dtype: object
14749    Sex and the City 2 (2010)
Name: title, dtype: object
3915    Finding Forrester (2000)
Name: title, dtype: object
168    Hackers (1995)
Name: title, dtype: object
12204    Then She Found Me (2007)
Name: title, dtype: object
12934    Marley & Me (2008)
Name: title, dtype: object
